<a href="https://colab.research.google.com/github/reolingovender/MIT805/blob/main/Word_Count_Analysis_PySpark_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

In [ ]:
pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import datasets
from datasets import load_dataset, load_from_disk
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, MapType, IntegerType, MapType, LongType, BooleanType, FloatType
from pyspark.sql.functions import col, explode, expr, collect_list, when
from pyspark.sql.functions import pandas_udf, PandasUDFType, col

In [ ]:
# Initialize Spark session
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

spark = SparkSession.builder.appName("ResourceOptimizedSession").master("local[*]").config("spark.executor.memory", "100g").config("spark.driver.memory", "50g").config("spark.executor.cores", "8").config("spark.sql.shuffle.partitions", "200").getOrCreate()

In [ ]:
def word_count(year):
  datasets.logging.set_verbosity_error()
  dataset = load_from_disk("/content/drive/MyDrive/MIT805/Merged/Books_{}".format(year))

  df = dataset.to_pandas()
  df['categories'] = df['categories'].apply(lambda x: ', '.join(x) if len(x) > 0 else '').astype(str)
  df = df[['rating', 'title', 'text', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase', 'book_title', 'price', 'store', 'categories']]

  # Convert Pandas DataFrames to PySpark DataFrames
  pyspark_df_reviews = spark.createDataFrame(df)

  # Convert 'timestamp' to correct formatting
  pyspark_df_reviews = pyspark_df_reviews.withColumn("timestamp", f.date_format(f.from_unixtime(col("timestamp") / 1000), "yyyy-MM-dd")).filter("cast(price as int) > 0").na.drop()

  pyspark_df_reviews = pyspark_df_reviews.withColumn("text", f.regexp_replace(col("text"), "[^a-zA-Z0-9 ]", ""))\
                                       .withColumn("word_count", f.size(f.split(col("text"), " ")))\
                                       .select('rating', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase','book_title', 'price', 'store', 'categories', 'word_count')

  pyspark_df_reviews.write.csv("/content/drive/MyDrive/MIT805/Results/Word_Count_Analysis/{}/".format(year), header=True, mode="overwrite")
  print("Year {} is done!".format(year))

In [ ]:
for year in range(2015, 2024):
  word_count(year)